<p style="color:#211f78"> <center><font size= 6> <b> COMPARACIÓN AUTOMATIZADA DE PRECIOS DE PRODUCTOS DE PRIMERA NECESIDAD EN LOS 3 SUPERMERCADOS MÁS POPULARES: METRO, TOTTUS y PLAZA VEA. </b> </font> </center> </p>

## DESCRIPCIÓN DEL CASO

Se quiere automatizar la comparación de precios de productos básicos de la canasta familiar
(arroz, azúcar, aceite, leche, etc) de tres supermercados que ofrecen sus productos a través de
su propio portal web. Estos supermercados son: Metro, Plaza Vea y Tottus.
Por lo que se pide escribir un programa en Python utilizando Jupyter notebook que recopile los
precios de un producto de la canasta básica familiar de 3 Supermercados anteriormente
mencionados y reporte todos sus artículos relacionados con la palabra clave ingresada en la
búsqueda, para que posteriormente pueda comparar los precios en los distintos
Supermercados de los productos exactamente iguales. Además, que se envíe
automáticamente al correo del usuario-consultor un reporte de precio más alto, precio más
bajo, productos en común comparando los precios entre los supermercados coincidentes.

# Importar librerías


In [227]:
import pandas as pd
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from pymongo import MongoClient
from tqdm import tqdm
from termcolor import colored
import csv
from bs4 import BeautifulSoup
import time
from webdriver_manager.microsoft import EdgeChromiumDriverManager
from webdriver_manager.chrome import ChromeDriverManager
import math

pd.set_option('max_colwidth', 2000)
                      
#jupyter notebook --NotebookApp.iopub_data_rate_limit=1.0e10

# Documentación de la nueva actualización de Selenium


In [293]:
ID = "id"
NAME = "name"
XPATH = "xpath"
LINK_TEXT = "link text"
PARTIAL_LINK_TEXT = "partial link text"
TAG_NAME = "tag name"
CLASS_NAME = "class name"
CSS_SELECTOR = "css selector"

# find_element(By.ID, "id")
# find_element(By.NAME, "name")
# find_element(By.XPATH, "xpath")
# find_element(By.LINK_TEXT, "link text")
# find_element(By.PARTIAL_LINK_TEXT, "partial link text")
# find_element(By.TAG_NAME, "tag name")
# find_element(By.CLASS_NAME, "class name")
# find_element(By.CSS_SELECTOR, "css selector")

# Ingresar producto y correo 

In [379]:
busqueda=str(input("Coloque el producto a buscar:"))
para_email= input("Enviar comparación de productos a: ") # Email de destino

Coloque el producto a buscar:leche
Enviar comparación de productos a: 20200339@lamolina.edu.pe


<p style="color:#deb03e"> <font size=6.5> <b> Programa para obtener los productos y precios de supermercados Metro (Usuario Jorge) </b> </font> </p>  

In [300]:
urls='https://www.metro.pe'          # URL de la pagina de Metro                   
options = webdriver.EdgeOptions()    #opciones del buscador 
options.add_argument('--incognito')  # Opcion de abrir ventana de Edge en modo incognito
options.add_argument("--start-maximized") # ventana maximizada 
s = Service(EdgeChromiumDriverManager().install())                       
driver = webdriver.Edge(service=s, options = options) #,service=s
driver.implicitly_wait(4) 
#driver.maximize_window()
driver.get(urls)

time.sleep(2)

In [301]:
# Para cancelar las notificaciones de Metro (ventana emergente)
driver.find_element(By.XPATH, '/html/body/div[47]/div/div/div[2]/button[2]').click()
time.sleep(1)

In [302]:
# Hacer clic en "Si acepto", pagina emergente sobre "Políticas general de Cookies"
driver.find_element(By.XPATH, '/html/body/div[46]/div/div[3]/button[1]').click()
time.sleep(1)

In [303]:
# Hacer clic en la barra de búsqueda
driver.find_element(By.XPATH, '/html/body/header[1]/div/div[2]/div[2]/div[1]/div[1]/input').click()
time.sleep(1)

In [305]:
# Ingresar el producto en la barra de busqueda y realizar la busqueda
driver.find_element(By.XPATH, '/html/body/header[1]/div/div[2]/div[2]/div[1]/div[1]/input').send_keys(busqueda+ Keys.ENTER)
time.sleep(1)

In [306]:
# Cantidad de productos
page=driver.find_element(By.XPATH, '/html/body/div[27]/div/div[2]/div[1]/div[1]/span[2]').text
int(page[:-10])

315

In [311]:
# Automatización de scrolls (se correra las veces que sea necesario)
scrolls = 5 + 1            # Seran 5 scrolls automaticos
while True:                # Mientras esto sea verdad se realizará lo siguiente:
    scrolls -= 1           # Cuenta regresiva, se resta 1 cada ciclo
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight)") # Se desplazará hasta la parte inferior de la página 
    time.sleep(1)      # Cada scroll automático tendra un timelap de 1 segundo
    if scrolls < 0:     # Cuando llegue a cero segundos:
        time.sleep(2)   # Le damos 2 segundos a que la pagina carge los contenidos hasta el ultimo scroll automatico
        driver.execute_script("window.scrollBy(0,-750)") # Un scroll up (rueda hacia arriba) para mostrar la ultima fila de productos
        elements = driver.find_elements(By.XPATH,'//li[@layout="19ccd66b-b568-43cb-a106-b52f9796f5cd"]') # Encontrar los productos de la pagina de acuerdo al XPATH
        break           # Se cierra el ciclo

        
print(colored('Productos extraidos:','red'))  
print(len(elements))        # Longitud de los elementos (productos) encontrados hasta coindicir con el total de productos
print('\n')
print(colored('Se llegó a extraer todos los productos?','yellow'))  
print(len(elements) >= int(page[:-10])) # Comparación 

Productos extraidos:
315


Se llegó a extraer todos los productos?
True


In [312]:
# Almacenar productos y precios 
productos =  []  # lista vacia
precios = []     # lista vacia
supermercado = []
for element in tqdm(elements):  # tqdm:Contar iteraciones, indicar el tiempo que tarda por iteración y el tiempo total transcurrido.  
        producto = element.find_element(By.XPATH,'.//a[@class="product-item__name"]').text  # Buscar producto en base al XPATH y mostrar el texto
        precio = element.find_element(By.XPATH,'.//span[@class="product-prices__value product-prices__value--best-price"]').text # Buscar precio en base al XPATH y mostrar el texto
        productos.append(producto)    # Agregar a la lista productos
        precios.append(precio)  # Agregar a la lista precios
        supermercado.append('Metro')


100%|██████████| 315/315 [00:43<00:00,  7.31it/s]


In [313]:
#Exportacion de productos y precios a un archivo excel 
dic= {'Supermercado': supermercado ,'Producto': productos , "Precio": precios}  # Crear diccionario con los values productos y precios (listas)
df = pd.DataFrame(dic , columns= ['Supermercado', "Producto", "Precio"] ) # Convertir a dataframe con columnas de etiquetas Producto y Precio
df.to_excel('metro.xlsx', index=False,encoding='cp1252')  # Exportar a excel

# Lectura y muestra de resultados
print(colored('Imprimiendo resultados.','blue'))  
pd.read_excel('metro.xlsx')

Imprimiendo resultados.


,Supermercado,Producto,Precio
0,Metro,Leche Evaporada Entera Gloria Pack de 6un 400g,S/21.60
1,Metro,Leche UHT Gloria Sin Lactosa tripack 1 L,S/16.20
2,Metro,Leche Entera Lata 395 g Pack 6 unid,S/21.40
3,Metro,Leche UHT Entera Cuisine & Co Caja 1 lt,S/. 4.75
4,Metro,Leche UHT Entera Monteagle Caja 1 Litro,S/. 3.90
5,Metro,Leche Entera UHT Gloria Tripack 900 ml c/u,S/11.50
6,Metro,Leche Concentrada Laive Bolsitarro 400g,S/2.90
7,Metro,Leche Semidescremada UHT Gloria Sin Lactosa Caja 1 L,S/5.60
8,Metro,Leche Evaporada Entera Ideal Lata 395 g,S/3.80
9,Metro,Leche Condensada Gloria Doy Pack 200 g,S/4.40


In [314]:
driver.close()  # Cerrar chromedriver

<p style="color:#059a38"> <font size=6.5> <b> Programa para obtener los productos y precios de supermercados Tottus (Usuario Christian) </b> </font> </p>   <font size=6.5> <b> 

In [320]:
urls='https://www.tottus.com.pe/'
options = webdriver.EdgeOptions()    #opciones del buscador 
options.add_argument('--incognito')  # Opcion de abrir ventana de Edge en modo incognito
options.add_argument("--start-maximized") # ventana maximizada 
s = Service(EdgeChromiumDriverManager().install())                       
driver = webdriver.Edge(service=s, options = options) #,service=s
driver.implicitly_wait(4) 
#driver.maximize_window()
driver.get(urls)

time.sleep(2)

In [321]:
#Al iniciar y entrar a la pagina de tottus nos salta un anuncio donde tottus nos quiere informar sobre sus mejores ofertas,
#con este codigo automatizamos dando click en cancelar. 
WebDriverWait(driver, 20)\
    .until(EC.element_to_be_clickable((By.CSS_SELECTOR,
                                      'button.align-right secondary slidedown-button'.replace(' ', '.'))))\
    .click()

In [322]:
# Cerrando ventana de mensaje de anuncios de oferta por agosto
driver.find_element(By.XPATH, '/html/body/div[3]/div/div/section/div[1]/div/button').click()
time.sleep(1)

In [323]:
# Configurando click para búsqueda de productos
driver.find_element(By.XPATH, '/html/body/div[1]/div[2]/div[2]/div[1]/div[1]/section/div[1]/div/input').click()
time.sleep(1)

In [324]:
# Ingresando producto 
driver.find_element(By.XPATH, '/html/body/div[1]/div[2]/div[2]/div[1]/div[1]/section/div[1]/div/input').send_keys(busqueda + Keys.ENTER)
time.sleep(1)

In [335]:
#Obtener el número de paginas
page = driver.find_element(By.XPATH, '/html/body/div[1]/section/div[1]/section/div[2]/div[1]/div[2]/section/div[1]/div[1]/p').text
pag = int(page[1:4])/47.5
import math
paginas = math.ceil(pag)  # Redondeo arriba
paginas

8

In [337]:
list = []
productos = []
precios =[]
supermercado = []
n=1
while n <= int(pag):
    driver.get(f'https://www.tottus.com.pe/buscar?q={busqueda}&page={n}')
    time.sleep(2)
    lista = driver.find_elements(By.XPATH,'//li[@class="product"]')
    for element in tqdm(lista):
        name = element.find_element(By.XPATH,'.//h2[@class="jsx-369422875 name title mini"]').text
        price = element.find_element(By.XPATH,'.//span[@class="jsx-4019956066 "]').text
        productos.append(name)
        precios.append(price)
        supermercado.append('Tottus')

    lista=list.append(lista)
    n += 1

print(colored('Se llegó a extraer todos los productos?','green'))  
print(len(productos) >= int(paginas)) # Comparación 

100%|██████████| 48/48 [00:03<00:00, 15.44it/s]

Se llegó a extraer todos los productos?
True


In [338]:
dic= {'Supermercado':supermercado , 'Producto': productos,"Precio": precios }
df = pd.DataFrame(dic , columns= ['Supermercado',"Producto","Precio"] )
df.to_excel('tottus.xlsx', index=False,encoding='cp1252')

# Lectura
tottus = pd.read_excel('tottus.xlsx')
tottus

,Supermercado,Producto,Precio
0,Tottus,SIX PACK MEZCLA LÁCTEA IDEAL AMANECER 6 UNIDADES,s/ 16.3 UN
1,Tottus,SIX PACK LECHE EVAPORADA IDEAL 395 GR,s/ 20.9 UN
2,Tottus,LECHE FRESCA ENTERA GLORIA 1 LT,s/ 5.7 UN
3,Tottus,LECHE EVAPORADA GLORIA 400 GR,s/ 4.4 UN
4,Tottus,PACK LECHE SEMIDESCREMADA GLORIA 3 UNIDADES,s/ 16.4 UN
5,Tottus,SIX PACK LECHE LIGHT GLORIA,s/ 22.4 UN
6,Tottus,LECHE FRESCA PASTEURIZADA,s/ 6.5 UN
7,Tottus,LECHE CHOCOLATADA GLORIA UHT 900 ML,s/ 5.2 UN
8,Tottus,LECHE EVAPORADA GLORIA LIGHT 400 GR,s/ 4.7 UN
9,Tottus,LECHE SIN LACTOSA LAIVE BOLSA 900 ML,s/ 4.5 UN


In [151]:
driver.close()  # Cerrar chromedriver

<p style="color:#fc0313"> <font size=6.5> <b> Programa para obtener los productos y precios de supermercados Plaza Vea  (Usuario Brian)  </b> </font> </p>   <font size=6.5> <b>

In [339]:
url='https://www.plazavea.com.pe'
options = webdriver.EdgeOptions()    #opciones del buscador 
options.add_argument('--incognito')  # Opcion de abrir ventana de Edge en modo incognito
options.add_argument("--start-maximized") # ventana maximizada 
s = Service(EdgeChromiumDriverManager().install())                       
driver = webdriver.Edge(service=s, options = options) #,service=s
driver.implicitly_wait(4) 
#driver.maximize_window()
driver.get(url)
time.sleep(2)

In [340]:
driver.find_element(By.XPATH, '/html/body/div[12]/div/div/div[2]/button[2]').click()

In [341]:
driver.find_element(By.XPATH, '/html/body/div[3]/header/div[2]/div/div[4]/section/div[2]/div/div/input').click()

In [342]:
driver.find_element(By.XPATH, '/html/body/div[3]/header/div[2]/div/div[4]/section/div[2]/div[2]/div[1]/input').send_keys(busqueda + Keys.ENTER)

In [343]:
page=driver.find_element(By.XPATH, '/html/body/div[4]/main/div/div[1]/div[2]/div[3]/div/div[1]/div/h2/span[2]').text
pag = int(page[1:4])/46
import math
paginas = math.ceil(pag)  # Redondeo arriba
paginas

5

In [344]:
list = []
productos = []
precios =[]
supermercado = []
n=1

while n <= paginas:
    driver.find_element(By.CSS_SELECTOR, f'#category-page > div.vitrine.resultItemsWrapper > div > div.paginator-container > div > div > span:nth-child({n})').click()
    time.sleep(3)
    lista = driver.find_elements(By.XPATH,'//div[@data-stock="true"]')
    for element in tqdm(lista):
        name = element.find_element(By.XPATH,'.//a[@class="Showcase__name"]').text
        price = element.find_element(By.XPATH,'.//span[@class="price"]').text
        productos.append(name)
        precios.append(price)
        supermercado.append('Plaza Vea')
    list.append(lista)
    n += 1   

100%|██████████| 5/5 [00:00<00:00, 14.85it/s]


In [345]:
dic= {'Supermercado':supermercado , 'Producto': productos,"Precio": precios }
df = pd.DataFrame(dic , columns= ['Supermercado',"Producto","Precio"] )
df.to_excel('plazavea.xlsx', index=False,encoding='cp1252')
# Lectura
plazavea=pd.read_excel('plazavea.xlsx')
plazavea

,Supermercado,Producto,Precio
0,Plaza Vea,Leche Evaporada GLORIA Light Lata 400g Paquete...,S/ 21.60 un
1,Plaza Vea,Leche Evaporada GLORIA sin Lactosa Lata 400g P...,S/ 22.80 un
2,Plaza Vea,Leche GLORIA Evaporada Entera Lata 400g Paquet...,S/ 21.60 un
3,Plaza Vea,Leche GLORIA Evaporada Niños Lata 400g Paquete...,S/ 21.60 un
4,Plaza Vea,Leche Sin Lactosa LAIVE Botella 400g Paquete 6...,S/ 21.90 un
5,Plaza Vea,Leche GLORIA UHT Entera Bolsa 900ml Paquete 3u...,S/ 11.70 un
6,Plaza Vea,Leche UHT GLORIA Light Pack 3un x 1L,S/ 15.70 un
7,Plaza Vea,Leche UHT LAIVE Sin Lactosa Caja 1L Paquete 4u...,S/ 20.30 un
8,Plaza Vea,Leche UHT GLORIA Entera Pack 3un x 1L,S/ 15.10 un
9,Plaza Vea,Leche UHT GLORIA sin Lactosa Pack 3un x 1L,S/ 16.20 un


In [346]:
driver.close()  # Cerrar chromedriver

# Comparar productos de acuerdo a un búsqueda en común (FASE de prueba)

In [375]:
df_metro = metro[metro['Producto'].str.count(r'(Leche.*Gloria.*400.*)')==1].sort_values(by='Precio', ascending = False)
df_metro=df_metro.reset_index(drop=True) 
df_metro

,Supermercado,Producto,Precio
0,Metro,Leche Concentrada Sin Lactosa Gloria Lata 400 gr,S/4.50
1,Metro,Leche Evaporada Semidescremada Light Gloria Lata 400 gr,S/4.30
2,Metro,Leche Evaporada Niños Gloria Lata 400 gr,S/4.20
3,Metro,Leche Evaporada Entera Gloria Lata 400 gr,S/3.95
4,Metro,Leche Evaporada Descremada Gloria Pack de 6 unid Lata 400 gr,S/26.50
5,Metro,Leche Concentrada Sin Lactosa Gloria Pack de 6 unid Lata 400 gr,S/23.30
6,Metro,Leche Evaporada Semidescremada Light Gloria Pack de 6 unid Lata 400 gr,S/21.90
7,Metro,Leche Evaporada Niños Gloria Pack de 6 unid Lata 400 gr,S/21.70
8,Metro,Leche Evaporada Entera Gloria Pack de 6un 400g,S/21.60


In [376]:
df_tottus = tottus[tottus['Producto'].str.count(r'(LECHE.*GLORIA.*400.*)')==1].sort_values(by='Precio', ascending = False)
df_tottus=df_tottus.reset_index(drop=True) 
df_tottus

,Supermercado,Producto,Precio
0,Tottus,LECHE SIN LACTOSA GLORIA 400 GR,s/ 4.9 UN
1,Tottus,LECHE EVAPORADA GLORIA LIGHT 400 GR,s/ 4.7 UN
2,Tottus,LECHE EVAPORADA NIÑOS DEFENSA GLORIA 400 GR,s/ 4.5 UN
3,Tottus,LECHE EVAPORADA GLORIA 400 GR,s/ 4.4 UN
4,Tottus,SIX PACK LECHE EVAPORADA DESCREMADA GLORIA 400 GR,s/ 27.2 UN
5,Tottus,SIX PACK LECHE EVAPORADA NIÑOS DHA GLORIA 400 GR,s/ 22.4 UN\ns/ 23 UN


In [377]:
df_plazavea= plazavea[plazavea['Producto'].str.count(r'(Leche.*GLORIA.*400.*)')==1].sort_values(by='Precio', ascending = False) 
df_plazavea = df_plazavea.reset_index(drop=True)
df_plazavea

,Supermercado,Producto,Precio
0,Plaza Vea,Leche GLORIA Evaporada Sin Lactosa Lata 400g,S/ 4.50 un
1,Plaza Vea,Leche Evaporada GLORIA Light Lata 400g,S/ 4.30 un
2,Plaza Vea,Leche GLORIA Evaporada Entera Lata 400g,S/ 3.99 un
3,Plaza Vea,Leche GLORIA Evaporada Descremada Lata 400g Pa...,S/ 26.50 un
4,Plaza Vea,Leche Evaporada GLORIA sin Lactosa Lata 400g P...,S/ 22.80 un
5,Plaza Vea,Leche Evaporada GLORIA Light Lata 400g Paquete...,S/ 21.60 un
6,Plaza Vea,Leche GLORIA Evaporada Entera Lata 400g Paquet...,S/ 21.60 un
7,Plaza Vea,Leche GLORIA Evaporada Niños Lata 400g Paquete...,S/ 21.60 un


In [399]:
comparacion = pd.concat([df_metro,df_tottus,df_plazavea])
comparacion

,Supermercado,Producto,Precio
0,Metro,Leche Concentrada Sin Lactosa Gloria Lata 400 gr,S/4.50
1,Metro,Leche Evaporada Semidescremada Light Gloria Lata 400 gr,S/4.30
2,Metro,Leche Evaporada Niños Gloria Lata 400 gr,S/4.20
3,Metro,Leche Evaporada Entera Gloria Lata 400 gr,S/3.95
4,Metro,Leche Evaporada Descremada Gloria Pack de 6 unid Lata 400 gr,S/26.50
5,Metro,Leche Concentrada Sin Lactosa Gloria Pack de 6 unid Lata 400 gr,S/23.30
6,Metro,Leche Evaporada Semidescremada Light Gloria Pack de 6 unid Lata 400 gr,S/21.90
7,Metro,Leche Evaporada Niños Gloria Pack de 6 unid Lata 400 gr,S/21.70
8,Metro,Leche Evaporada Entera Gloria Pack de 6un 400g,S/21.60
0,Tottus,LECHE SIN LACTOSA GLORIA 400 GR,s/ 4.9 UN


In [426]:
# Exportando a excel y csv
writer = pd.ExcelWriter('Reporte.xlsx')
df1= pd.DataFrame(df_metro)
df2= pd.DataFrame(df_tottus)
df3= pd.DataFrame(df_plazavea)

df1.to_excel(writer, 'Metro', index=False)
df2.to_excel(writer, 'Tottus', index=False)
df3.to_excel(writer, 'Plaza Vea', index=False)

writer.save()
writer.close()

comparacion.to_csv('Reporte.csv', index=False,encoding='cp1252')

# Envío de correo automático (FASE de prueba)

In [425]:
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.base import MIMEBase
from email import encoders
 
# Iniciamos los parámetros del script
remitente = 'okoyekennethoptimizer@gmail.com'
destinatarios = para_email # Destinatario
asunto = '[RESULTADOS] TRABAJO FINAL LP2'
cuerpo = '''
Hola estimado usuario, le enviamos las comparaciones de los productos.
Integrantes:
- Tanta Medina, Christian Rodolfo 20200338
- Taquiri Pillaca, Jorge Ignacio 20200339
- Tay Saccaco, Brian 
'''
ruta_adjunto = 'Reporte.xlsx'
nombre_adjunto = 'comparacion de productos.xlsx'

mensaje = MIMEMultipart() # Creamos el objeto mensaje
 
# Establecemos los atributos del mensaje
mensaje['From'] = remitente
mensaje['To'] = ", ".join(destinatarios)  # . join en caso haya mas de 1 destinatario
mensaje['Subject'] = asunto
# Agregamos el cuerpo del mensaje como objeto MIME de tipo texto
mensaje.attach(MIMEText(cuerpo, 'plain')) 
# Abrimos el archivo que vamos a adjuntar
archivo_adjunto = open(ruta_adjunto, 'rb') 
# Creamos un objeto MIME base 
adjunto_MIME = MIMEBase('application', 'octet-stream') 
# Cargamos el archivo adjunto
adjunto_MIME.set_payload((archivo_adjunto).read())
# Codificamos el objeto en BASE64
encoders.encode_base64(adjunto_MIME)
# Agregamos una cabecera al objeto
adjunto_MIME.add_header('Content-Disposition', "attachment; filename= %s" % nombre_adjunto) 
# Finalmente lo agregamos al mensaje
mensaje.attach(adjunto_MIME) 
# Creamos la conexión con el servidor
sesion_smtp = smtplib.SMTP('smtp.gmail.com', 587) 
# Ciframos la conexión
sesion_smtp.starttls() 
# Iniciamos sesión en el servidor
sesion_smtp.login('okoyekennethoptimizer@gmail.com','nmmrjdphptwfgjgb') 
# Convertimos el objeto mensaje a texto
texto = mensaje.as_string() 
# Enviamos el mensaje
sesion_smtp.sendmail(remitente, destinatarios, texto)
# Cerramos la conexión
sesion_smtp.quit() 

(221,
 b'2.0.0 closing connection x140-20020a1f3192000000b00378a524ac44sm4526344vkx.30 - gsmtp')

# Envio

![foto](captura1.png "")

[](usuario.png)